# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather = pd.read_csv("weather.csv")
print(weather)
print(weather.info())

                City Country      Lat      Long   Temp  Humidity  Cloudiness  \
0          Busselton      AU -33.6500  115.3333  63.59        59           0   
1         The Valley      AI  18.2170  -63.0578  82.26        76          18   
2              Bluff      NZ -46.6000  168.3333  53.80        62          15   
3          Ketchikan      US  55.3422 -131.6461  55.87        94          75   
4    Arraial do Cabo      BR -22.9661  -42.0278  70.77        84         100   
..               ...     ...      ...       ...    ...       ...         ...   
566          Honiara      SB  -9.4333  159.9500  86.47        66          14   
567    Partizanskoye      RU  55.5006   94.3900  38.03        76         100   
568      Henties Bay     NaN -22.1160   14.2845  54.86        81           0   
569    Porto-Vecchio      FR  41.5910    9.2795  65.84        77           0   
570            Bilma      NE  18.6853   12.9164  85.86        13          95   

     Wind Speed  
0          2.42  
1  

In [10]:
filepath = os.path.join('weather.csv')
weather2_df = pd.read_csv(filepath)
weather2_df.head()

,City,Country,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed
0,Busselton,AU,-33.6500,115.3333,63.59,59,0,2.42
1,The Valley,AI,18.2170,-63.0578,82.26,76,18,13.27
2,Bluff,NZ,-46.6000,168.3333,53.80,62,15,19.30
3,Ketchikan,US,55.3422,-131.6461,55.87,94,75,12.66
4,Arraial do Cabo,BR,-22.9661,-42.0278,70.77,84,100,28.19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Access maps with unique API key

src="https://maps.googleapis.com/maps/api/js?key=api_keys"


locations = weather2_df[['Lat', 'Long']]
humidity = weather2_df['Humidity']
intensity = humidity.max()

# Plot heat map
figure = gmaps.figure(center=(51.5, 0.1), zoom_level=2)

# Make a heat layer
layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=intensity, point_radius=3)

# Add the layer to the heat map
figure.add_layer(layer)

# Display
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* Max temp lower than 80 but higher than 70
* wind speed less than 10
* zero cloudiness
* drop any ros that do not contain all 3 conditions, I want an Ideal weather

In [13]:
criteria_df = weather2_df[(weather2_df['Temp'] > 70)
                          &(weather2_df['Temp'] < 80)
                          &(weather2_df['Wind Speed'] < 10)
                          &(weather2_df['Cloudiness'] < 20)].dropna()

                         
criteria_df.head()


,City,Country,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed
53,Faanui,PF,-16.4833,-151.7500,78.30,72,10,5.14
81,Sinazongwe,ZM,-17.2614,27.4618,71.46,35,6,1.68
123,Kiryat Gat,IL,31.6100,34.7642,73.62,76,0,1.45
125,Vao,NC,-22.6667,167.4833,71.94,78,6,8.63
159,Buraidah,SA,26.3260,43.9750,75.65,24,1,3.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = criteria_df.reset_index(drop=True)
hotel_df["Hotel Name"] = " "
hotel_df

# Make a params dictionary to update each iteration
params = {
    'radius': 50000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

hotel_df

,City,Country,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Faanui,PF,-16.4833,-151.7500,78.30,72,10,5.14,
1,Sinazongwe,ZM,-17.2614,27.4618,71.46,35,6,1.68,
2,Kiryat Gat,IL,31.6100,34.7642,73.62,76,0,1.45,
3,Vao,NC,-22.6667,167.4833,71.94,78,6,8.63,
4,Buraidah,SA,26.3260,43.9750,75.65,24,1,3.76,
5,Riyadh,SA,24.6877,46.7219,77.14,18,0,8.39,
6,Tevaitoa,PF,-16.7833,-151.5000,78.69,71,1,6.11,
7,Angoche,MZ,-16.2325,39.9086,71.94,87,11,0.72,
8,Carnarvon,AU,-24.8667,113.6333,71.67,33,0,4.61,
9,Nísia Floresta,BR,-6.0911,-35.2086,79.18,91,6,5.64,


In [20]:
for index, row in hotel_df.iterrows():
    
    params['location'] = f"{row['Lat']},{row['Long']}"

    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    response = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')

hotel_df

,City,Country,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Faanui,PF,-16.4833,-151.7500,78.30,72,10,5.14,InterContinental Bora Bora Resort Thalasso Spa
1,Sinazongwe,ZM,-17.2614,27.4618,71.46,35,6,1.68,Masumu River Lodge
2,Kiryat Gat,IL,31.6100,34.7642,73.62,76,0,1.45,King David Hotel
3,Vao,NC,-22.6667,167.4833,71.94,78,6,8.63,OURE TERA beach resort
4,Buraidah,SA,26.3260,43.9750,75.65,24,1,3.76,Best Western Plus Buraidah
5,Riyadh,SA,24.6877,46.7219,77.14,18,0,8.39,Four Seasons Hotel Riyadh At Kingdom Center
6,Tevaitoa,PF,-16.7833,-151.5000,78.69,71,1,6.11,InterContinental Bora Bora Resort Thalasso Spa
7,Angoche,MZ,-16.2325,39.9086,71.94,87,11,0.72,Hotel Chando Angoche
8,Carnarvon,AU,-24.8667,113.6333,71.67,33,0,4.61,Carnarvon Motel
9,Nísia Floresta,BR,-6.0911,-35.2086,79.18,91,6,5.64,SERHS Natal


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [23]:
src="https://maps.googleapis.com/maps/api/js?key=api_keys"


locations = weather2_df[['Lat', 'Long']]
humidity = weather2_df['Humidity']
intensity = humidity.max()

# Plot heat map
figure = gmaps.figure(center=(51.5, 0.1), zoom_level=2)

layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=intensity, point_radius=3)

figure.add_layer(layer)

# Display
figure

# Add marker layer ontop of heat map
layer = gmaps

# Display figure


Figure(layout=FigureLayout(height='420px'))